Copyright 2021-2023 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of MindSPONGE:
MindSpore Simulation Package tOwards Next Generation molecular modelling.

MindSPONGE is open-source software based on the AI-framework:
MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and
limitations under the License.

MindSPONGE advanced tutorial 04: Hybrid enhanced sampling and MetaITS

In [1]:
from mindspore import context
from mindsponge import Sponge
from mindsponge import ForceField
from mindsponge import set_global_units
from mindsponge import Protein
from mindsponge import WithEnergyCell
from mindsponge import UpdaterMD
from mindsponge.optimizer import SteepestDescent
from mindsponge.control import VelocityVerlet
from mindsponge.callback import WriteH5MD, RunInfo
from mindsponge.control import Langevin
from mindsponge.sampling import Metadynamics, ITS
from mindsponge.function import VelocityGenerator
from mindsponge.colvar import Torsion
from mindsponge.function import PI

[WARNING] ME(190458:140013692118272,MainProcess):2023-03-24-17:55:32.801.147 [mindspore/run_check/_check_version.py:75] Can not found cuda libs, please confirm that the correct cuda version has been installed, you can refer to the installation guidelines: https://www.mindspore.cn/install


In [2]:
context.set_context(mode=context.PYNATIVE_MODE, device_target="GPU")
set_global_units('nm', 'kj/mol')

In [3]:
system = Protein(pdb='alad.pdb')
potential = ForceField(system, 'AMBER.FF14SB')

In [4]:
phi = Torsion([4, 6, 8, 14])
psi = Torsion([6, 8, 14, 16])

In [5]:
min_opt = SteepestDescent(system.trainable_params(), 1e-7)
mini = Sponge(system, potential, min_opt, metrics={'phi': phi, 'psi': psi})

[MindSPONGE] Using fully connected neighbour list (not updated).


In [6]:
run_info = RunInfo(10)
mini.run(100, callbacks=[run_info])

[MindSPONGE] Started simulation at 2023-03-24 17:55:42
[MindSPONGE] Step: 0, E_pot: -55.75643, phi: -3.140886, psi: -3.1410766
[MindSPONGE] Step: 10, E_pot: -58.41481, phi: -3.1339903, psi: -3.1368215
[MindSPONGE] Step: 20, E_pot: -60.197777, phi: -3.1276336, psi: -3.1337032
[MindSPONGE] Step: 30, E_pot: -61.60443, phi: -3.1219037, psi: -3.1313035
[MindSPONGE] Step: 40, E_pot: -62.77914, phi: -3.1167603, psi: -3.1294105
[MindSPONGE] Step: 50, E_pot: -63.786407, phi: -3.1121268, psi: -3.1279025
[MindSPONGE] Step: 60, E_pot: -64.66443, phi: -3.107924, psi: -3.126701
[MindSPONGE] Step: 70, E_pot: -65.43921, phi: -3.1040812, psi: -3.125749
[MindSPONGE] Step: 80, E_pot: -66.13011, phi: -3.100538, psi: -3.125002
[MindSPONGE] Step: 90, E_pot: -66.75129, phi: -3.0972445, psi: -3.1244242
[MindSPONGE] Finished simulation at 2023-03-24 17:55:46
[MindSPONGE] Simulation time: 4.04 seconds.
--------------------------------------------------------------------------------


In [7]:
metad = Metadynamics(
    colvar=[phi, psi],
    update_pace=10,
    height=2.5,
    sigma=0.05,
    grid_min=-PI,
    grid_max=PI,
    grid_bin=360,
    temperature=300,
    bias_factor=100,
)

In [8]:
its = ITS(
    sim_temp=300,
    temp_min=270,
    temp_max=670,
    temp_bin=200,
    update_pace=100,
    unlinear_temp=True,
)

In [9]:
sim = WithEnergyCell(system, potential, bias=metad, wrapper=its)

[MindSPONGE] Using fully connected neighbour list (not updated).


In [10]:
vgen = VelocityGenerator(300)
velocity = vgen(system.coordinate.shape, system.atom_mass)
opt = UpdaterMD(
    system,
    integrator=VelocityVerlet(system),
    thermostat=Langevin(system, 300),
    time_step=1e-3,
    velocity=velocity
)

In [11]:
md = Sponge(sim, optimizer=opt, metrics={'phi': phi, 'psi': psi})

In [12]:
cb_h5md = WriteH5MD(system, 'tutorial_a04.h5md', save_freq=10)
md.run(1000, callbacks=[run_info, cb_h5md])

[MindSPONGE] Started simulation at 2023-03-24 17:55:53
[MindSPONGE] Step: 0, E_pot: -80.015175, E_kin: 111.25446, E_tot: 31.239288, Temperature: 424.78848, phi: -3.090533, psi: -3.1262357
[MindSPONGE] Step: 10, E_pot: -94.96431, E_kin: 48.351128, E_tot: -46.613182, Temperature: 184.61284, phi: -3.0379052, psi: 3.1247265
[MindSPONGE] Step: 20, E_pot: -104.170815, E_kin: 52.031548, E_tot: -52.139267, Temperature: 198.6653, phi: -2.9780364, psi: -3.1205432
[MindSPONGE] Step: 30, E_pot: -105.9071, E_kin: 47.039566, E_tot: -58.86753, Temperature: 179.60506, phi: -2.8954506, psi: -3.0621872
[MindSPONGE] Step: 40, E_pot: -113.51082, E_kin: 64.22586, E_tot: -49.284958, Temperature: 245.22525, phi: -2.8580468, psi: -3.0479946
[MindSPONGE] Step: 50, E_pot: -123.635796, E_kin: 77.99435, E_tot: -45.64145, Temperature: 297.7957, phi: -2.84861, psi: -3.0790868
[MindSPONGE] Step: 60, E_pot: -112.17632, E_kin: 69.66453, E_tot: -42.511795, Temperature: 265.99103, phi: -2.834069, psi: 3.1268237
[MindSPO